# Customer Churn Prediction
---
In this notebook, we will build and evaluate a machine learning model to predict customer churn. We'll use a Random Forest classifier and explore hyperparameter tuning to optimize its performance.

## 1. Import Libraries

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

## 2. Load Data

In [ ]:
# Load your dataset (replace 'your_dataset.csv' with your actual file)
df = pd.read_csv('../data/processed/processed_customer_data.csv')

## 3. Data Preprocessing

In [ ]:
# Data Preprocessing
# Assuming 'CustomerID', 'SubscriptionStart', 'SubscriptionEnd', 'LastPurchaseDate', and 'AgeGroup' are not needed for modeling
X = df.drop(['CustomerID', 'Churn', 'SubscriptionStart', 'SubscriptionEnd', 'LastPurchaseDate', 'AgeGroup'], axis=1)
y = df['Churn']  # Target variable

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 4. Model Building & Evaluation

In [ ]:
# Model Building - Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Predictions
y_pred = rf_model.predict(X_test_scaled)

# Model Evaluation
print("Classification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

## 5. Feature Importance

In [ ]:
# Feature Importance
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': rf_model.feature_importances_})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\nTop 10 Feature Importance:\n", feature_importance.head(10))

## 6. Hyperparameter Tuning

In [ ]:
# Hyperparameter Tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

print("\nBest Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

## 7. Evaluate Best Model

In [ ]:
# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_scaled)
print("\nClassification Report for Best Model:\n", classification_report(y_test, y_pred_best))